In [45]:
import nltk
import json
import re
import string
import sys
#pip install language-tool-python
import language_tool_python
#pip install truecase
import truecase

In [46]:
def total_word(text): 
    words = text.split()
    # remove punctuation from each word
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table).lower() for w in words]
    return(list(stripped))


In [47]:
def truecasing_by_sentence_segmentation(input_text):
    # split the text into sentences
    sentences = sent_tokenize(input_text, language='english')
    # capitalize the sentences
    sentences_capitalized = [s.capitalize() for s in sentences]
    # join the capitalized sentences
    text_truecase = re.sub(" (?=[\.,'!?:;])", "", ' '.join(sentences_capitalized))
    return text_truecase


In [48]:
#split the text by words and remove punctuation
def split_remove_punc(text): 
    words = text.split()
    # remove punctuation from each word
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in words]
    return(stripped)

In [49]:
#Find grammar and spelling errors using language_tool_python
def get_total_errors(text): 
    try:
        tool = language_tool_python.LanguageTool()
        matches = tool.check(text)
        errors=len(matches)
        return errors
    except:
        return 0

In [50]:
#This function input the cleaned text, and return ratio of wrong capitalizations.
def get_cap_problems(text):
    try:
        #using packages to find right casing of the text
        fixed_text=truecase.get_true_case(text)
        result=truecasing_by_sentence_segmentation(fixed_text)
        #compare the truecase text with the original list of words
        #get list c, which is the list of the different wording choices
        a=split_remove_punc(result)
        b=split_remove_punc(text)
        s=set(b)
        c=[x for x in a if x not in s]
        #calculate the ratio of wrong capitalization in the text
        #number of words that used wrong casing / number of total words
        total_words=total_word(text)
        return "{:.3f}".format(float(len(c)/len(total_words)))
    except:
        return 0      

In [51]:
#add the two attributes to the single json file specified by filepath
def add_language_style(filepath):
    file=open(filepath,'r')
    content=json.load(file)
    # Get cleaned letter content
    try:
        text=content["X-TIKA:content"].strip('\n')
        #tokenize the words in content
        token=total_words(text)
        content['total_errors']=get_total_errors(text)
        content['wrong_cap_ratio']=get_cap_problems(text)
    except:
        pass
    file =open("filepath", "w+")
    file.write(json.dumps(content))
    file.close()

for filepath in listdir(...):
    add_language_style(filepath)